In [1]:
import time
import torch
from torch.distributions import Categorical, kl
from d2l.torch import Animator

from net import TransformerModel
from aco import ACO
from utils import gen_instance, reformat, load_test_dataset

torch.manual_seed(1234)

EPS = 1e-10
device = 'cpu'

In [2]:
def infer_instance(model, price, weight, n_ants, t_aco_diff):
    if model:
        model.eval()
        src = reformat(price, weight)
        phe_vec, heu_vec = model(src)
        phe_vec, heu_vec = phe_vec.squeeze() + EPS, heu_vec.squeeze() + EPS
        aco = ACO(
            price=price,
            weight=weight,
            n_ants=n_ants,
            pheromone=phe_vec,
            heuristic=heu_vec,
            device=device
        )
    else:
        aco = ACO(
            price=price,
            weight=weight,
            n_ants=n_ants,
            device=device
        )
        
    results = torch.zeros(size=(len(t_aco_diff),), device=device)
    for i, t in enumerate(t_aco_diff):
        best_cost, _ = aco.run(t)
        results[i] = best_cost
    return results
    
@torch.no_grad()
def test(dataset, model, n_ants, t_aco):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco_diff),), device=device)
    start = time.time()
    for price, weight in dataset:
        results = infer_instance(model, price, weight, n_ants, t_aco_diff)
        sum_results += results
    end = time.time()
    
    return sum_results / len(dataset), end-start

### Test on MKP50

MetaACO

In [4]:
n_ants = 20
n_node = 50
t_aco = [1, 2, 3, 4, 5, 10]
test_list = load_test_dataset(n_node, device)
net_mkp = TransformerModel().to(device)
net_mkp.load_state_dict(torch.load(f'../pretrained/mkp/mkp{n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_mkp, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i]))    

total duration:  748.6528759002686
T=1, average obj. is 11.364100456237793.
T=2, average obj. is 11.379688262939453.
T=3, average obj. is 11.386866569519043.
T=4, average obj. is 11.389361381530762.
T=5, average obj. is 11.389361381530762.
T=10, average obj. is 11.391247749328613.


In [ ]:
n_ants = 20
n_node = 50
t_aco = [1, 2, 3, 4, 5, 10]
test_list = load_test_dataset(n_node, device)
avg_aco_best, duration = test(test_list, None, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i]))    

total duration:  678.2525622844696
T=1, average obj. is 10.16207504272461.
T=2, average obj. is 10.767912864685059.
T=3, average obj. is 11.011064529418945.
T=4, average obj. is 11.147146224975586.
T=5, average obj. is 11.235312461853027.
T=10, average obj. is 11.441211700439453.


### Test on MKP100

MetaACO

In [5]:
n_ants = 20
n_node = 100
t_aco = [1, 2, 3, 4, 5, 10]
test_list = load_test_dataset(n_node, device)
net_mkp = TransformerModel().to(device)
net_mkp.load_state_dict(torch.load(f'../pretrained/mkp/mkp{n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_mkp, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i]))    

total duration:  2580.5782539844513
T=1, average obj. is 20.47673797607422.
T=2, average obj. is 20.524988174438477.
T=3, average obj. is 20.54534339904785.
T=4, average obj. is 20.563329696655273.
T=5, average obj. is 20.577308654785156.
T=10, average obj. is 20.616466522216797.


ACO

In [ ]:
n_ants = 20
n_node = 100
t_aco = [1, 2, 3, 4, 5, 10]
test_list = load_test_dataset(n_node, device)
avg_aco_best, duration = test(test_list, None, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average obj. is {}.".format(t, avg_aco_best[i]))    

total duration:  2164.118937253952
T=1, average obj. is 17.052200317382812.
T=2, average obj. is 18.419252395629883.
T=3, average obj. is 19.121368408203125.
T=4, average obj. is 19.442798614501953.
T=5, average obj. is 19.726795196533203.
T=10, average obj. is 20.378917694091797.
